In [1]:
"""
Intersect 2011 Thai flood depth raster (courtesy of JBA) with WRI powerplant point locations
Ballpark estimate of rehabilitation costs...
"""

'\nIntersect 2011 Thai flood depth raster (courtesy of JBA) with WRI powerplant point locations\nBallpark estimate of rehabilitation costs...\n'

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
import shapely
import snail.damages, snail.intersection

/home/fred/micromamba/envs/wb_flood/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/fred/micromamba/envs/wb_flood/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
raster_path = "TH_FLRF_ChaoPhraya2011_RD_01.tif"
powerplants_path = "powerplants.geoparquet"
damage_curve_path = "damage_curve.csv"

In [4]:
raster = rasterio.open(raster_path)
grid = snail.intersection.GridDefinition.from_raster(raster_path)
minx, miny, maxx, maxy = raster.bounds
raster_values = raster.read().squeeze()  # squeeze to drop leading size 1 dim
raster_values.shape

(15836, 8232)

In [5]:
global_pp = gpd.read_parquet(powerplants_path)
thai_pp = global_pp.cx[minx: maxx, miny: maxy]

In [6]:
indicies = thai_pp.apply(lambda row: snail.intersection.get_indices(row.geometry, grid), axis=1)
depths = snail.intersection.get_raster_values_for_splits(indicies, raster_values)
depths = depths[depths > 0]
depths.name = "depth_m"

In [7]:
curve = snail.damages.PiecewiseLinearDamageCurve.from_csv(
    damage_curve_path,
    intensity_col="depth_m",
    damage_col="damage_fraction"
)
damages = thai_pp.join(depths, how="inner")
damages["damage_ratio"] = curve.damage_fraction(damages.depth_m)

In [8]:
# top plants by damage
# only EGCO Cogen is of any meaningful size
damages.sort_values("damage_ratio", ascending=False).head()

# https://www.gfdrr.org/sites/default/files/publication/Thai_Flood_2011_2.pdf
# page 120:
# ... The privately owned 280MW combined-cycle power plant in Rojana Industrial Park was flooded in the
# third week of October and remained flooded at the time of the team’s field visit on 15 November, 2011.
# ... the damage to the Rojana power plant is estimated at THB 2.4 billion.
# 
# FYI that's about $65M
# however... the JBA flood map does not show flooding of the facility, so no damage for us...

,name,source_id,power_mw,primary_fuel,estimated_generation_gwh_2017,asset_type,geometry,depth_m,damage_ratio
21455,Ang Thong Solar Power Plant,WRI1026223,1.2,Solar,1.94,source,POINT (100.47610 14.58940),4.159156,0.300000
21557,Nakhon Sawan Solar Power Plant,WRI1026244,1.0,Solar,1.81,source,POINT (100.11860 15.73860),3.302053,0.300000
21486,EGCO Cogen,WRI1019467,112.0,Gas,489.78,source,POINT (100.57270 14.09770),3.963405,0.300000
21591,Sai Prapa Solar Power Plant,WRI1026303,7.2,Solar,13.18,source,POINT (100.20020 13.99990),2.566649,0.210332
21639,Wang Luek Solar Power Plant,WRI1026334,8.0,Solar,14.24,source,POINT (100.13310 14.73910),2.131103,0.174488


In [9]:
# EGCO Cogen plant has 112MW capacity
# what's a reconstruction cost estimate for the flooding there?

# US power plant construction costs
# https://www.eia.gov/electricity/generatorcosts/xls/generator_costs_2021.xlsx
# 920 [USD 2021] / capacity [kW]
# 920000 [USD 2021] / capacity [MW]

gas_cost_per_mw = 920000

gas = damages[damages.primary_fuel == "Gas"].copy()
gas["rehab_cost"] = gas.damage_ratio * gas_cost_per_mw * gas.power_mw

# ~$300k with extremely low confidence :) 

In [11]:
gas.sort_values("rehab_cost", ascending=False)

,name,source_id,power_mw,primary_fuel,estimated_generation_gwh_2017,asset_type,geometry,depth_m,damage_ratio,rehab_cost
21486,EGCO Cogen,WRI1019467,112.0,Gas,489.78,source,POINT (100.57270 14.09770),3.963405,0.300000,3.091200e+07
21462,Bang Bo,WRI1019483,350.0,Gas,1530.58,source,POINT (100.84160 13.49200),0.871847,0.071748,2.310277e+07
21456,B Grimm BIP,WRI1019488,230.0,Gas,1005.81,source,POINT (100.56490 13.97450),0.148729,0.011898,2.517682e+06
